In [1]:
import tensorflow as tf
import tflearn
import numpy as np

In [2]:
class LSTM:
    def __init__(self, vocabulary_size=200, num_nodes=128, batch_size=10):
        self.num_nodes = num_nodes
        self.reuse = False
        self.vocabulary_size = vocabulary_size
        self.num_nodes = num_nodes
        with tf.variable_scope("LSTM_var",reuse=self.reuse):
            with tf.variable_scope("input_gate"):
                self.ix = tf.get_variable("ix", [vocabulary_size, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.im = tf.get_variable("im", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.ib = tf.get_variable("ib", [num_nodes], 
                                      tf.float32, tf.constant_initializer(0))
            with tf.variable_scope("forget_gate"):
                self.fx = tf.get_variable("fx", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.fm = tf.get_variable("fm", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.fb = tf.get_variable("fb", [num_nodes], 
                                     tf.float32, tf.constant_initializer(0))
            
            with tf.variable_scope("output_gate"):
                self.ox = tf.get_variable("ox", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.om = tf.get_variable("om", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.ob = tf.get_variable("ob", [num_nodes], 
                                     tf.float32, tf.constant_initializer(0))
            
            with tf.variable_scope("memory_cell"):
                self.cx = tf.get_variable("cx", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.cm = tf.get_variable("cm", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.cb = tf.get_variable("cb", [num_nodes], 
                                     tf.float32, tf.constant_initializer(0))
                
            with tf.variable_scope("output_layer"):
                self.w = tf.get_variable("w", [num_nodes, vocabulary_size], tf.float32,
                                        tf.truncated_normal_initializer(stddev=0.1))
                self.b = tf.get_variable("b", [vocabulary_size], tf.float32,
                                        tf.constant_initializer(0))
            
            self.saved_output = tf.get_variable("saved_output", [batch_size, num_nodes], tf.float32,
                                        tf.truncated_normal_initializer(stddev=0.1), trainable=False)
            self.saved_state = tf.get_variable("saved_state", [batch_size, num_nodes], tf.float32,
                                        tf.truncated_normal_initializer(stddev=0.1), trainable=False)
            
        self.variables = tf.get_collection(tf.GraphKeys.VARIABLES, scope="LSTM_var")
        
    def lstm_cell(self, inputs, hidden_layer, state):         
        with tf.variable_scope("LSTM_CELL",reuse=self.reuse):
            input_gate = tf.sigmoid(tf.matmul(inputs, self.ix) + tf.matmul(hidden_layer, self.im) + self.ib,
                                    name="input_gate")
            forget_gate = tf.sigmoid(tf.matmul(inputs, self.fx) + tf.matmul(hidden_layer, self.fm) + self.fb,
                                    name="forget_gate")
            update_gate = tf.tanh(tf.matmul(inputs, self.cx) + tf.matmul(hidden_layer, self.cm) + self.cb,
                            name="update_gate")
            state = tf.add(tf.mul(forget_gate, state), tf.mul(input_gate, update_gate),
                           name="state")
            output_gate = tf.sigmoid(tf.matmul(inputs, self.ox) + tf.matmul(hidden_layer, self.om) + self.ob,
                                     name="output_gate") 
            output = tf.mul(output_gate, tf.tanh(state), name="output")
            
        return output, state   
    
    def model(self):
        pass
    
    def __call__(self, inputs):
        self.model(inputs)
        

In [3]:
L = LSTM()

In [4]:
h = tf.Variable(tf.truncated_normal((1,128)))
state = tf.Variable(tf.truncated_normal((1,128)))
test_inputs = tf.placeholder(tf.float32, shape=[1, 200])

In [5]:
L.lstm_cell(test_inputs, h, state)

(<tf.Tensor 'LSTM_CELL/output:0' shape=(1, 128) dtype=float32>,
 <tf.Tensor 'LSTM_CELL/state:0' shape=(1, 128) dtype=float32>)

In [6]:
for i in L.variables:
    print("{},{}".format(i.name, i.get_shape()))

LSTM_var/input_gate/ix:0,(200, 128)
LSTM_var/input_gate/im:0,(128, 128)
LSTM_var/input_gate/ib:0,(128,)
LSTM_var/forget_gate/fx:0,(200, 128)
LSTM_var/forget_gate/fm:0,(128, 128)
LSTM_var/forget_gate/fb:0,(128,)
LSTM_var/output_gate/ox:0,(200, 128)
LSTM_var/output_gate/om:0,(128, 128)
LSTM_var/output_gate/ob:0,(128,)
LSTM_var/memory_cell/cx:0,(200, 128)
LSTM_var/memory_cell/cm:0,(128, 128)
LSTM_var/memory_cell/cb:0,(128,)
LSTM_var/output_layer/w:0,(128, 200)
LSTM_var/output_layer/b:0,(200,)
LSTM_var/saved_output:0,(10, 128)
LSTM_var/saved_state:0,(10, 128)
